In [1]:
import os

In [2]:
%pwd

'd:\\New Projects\\Customer Churn Prediction - End to End (ML)\\Customer-Churn-Prediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\New Projects\\Customer Churn Prediction - End to End (ML)\\Customer-Churn-Prediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    colsample_bytree: float
    learning_rate: float
    max_depth: int
    n_estimators: int
    subsample: float
    target_column: str

In [6]:
from CustomerChurn.constants import *
from CustomerChurn.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.XGBClassifier
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir= config.root_dir,
            train_data_path= config.train_data_path,
            test_data_path= config.test_data_path,
            model_name= config.model_name,
            colsample_bytree= params.colsample_bytree,
            learning_rate= params.learning_rate,
            max_depth= params.max_depth,
            n_estimators= params.n_estimators,
            subsample= params.subsample,
            target_column= schema.name
            
        )
        
        return model_trainer_config

In [8]:
import pandas as pd 
import os
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from CustomerChurn import logger

In [12]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        
        X_train = train_data.drop([self.config.target_column], axis=1)
        X_test = test_data.drop([self.config.target_column], axis=1)
        y_train = train_data[[self.config.target_column]]
        y_test = train_data[[self.config.target_column]]
        
        model = XGBClassifier(colsample_bytree=self.config.colsample_bytree, learning_rate=self.config.learning_rate,
                              max_depth=self.config.max_depth, n_estimators=self.config.n_estimators,
                              subsample=self.config.subsample)
        model.fit(X_train, y_train)
        
        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))
        logger.info("Model got built successfully")
        

In [13]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2024-09-20 13:48:43,352] 32 CustomerChurnLogger - common - INFO - yaml file: config\config.yaml loaded successfully
[2024-09-20 13:48:43,356] 32 CustomerChurnLogger - common - INFO - yaml file: params.yaml loaded successfully
[2024-09-20 13:48:43,362] 32 CustomerChurnLogger - common - INFO - yaml file: schema.yaml loaded successfully
[2024-09-20 13:48:43,366] 52 CustomerChurnLogger - common - INFO - Created directory at: artifacts
[2024-09-20 13:48:43,368] 52 CustomerChurnLogger - common - INFO - Created directory at: artifacts/model_trainer


[2024-09-20 13:48:43,926] 20 CustomerChurnLogger - 1762219418 - INFO - Model got built successfully
